In [ ]:
!pip3 install sklearn
!pip3 install tqdm
!pip3 install tensorboardX
!pip3 install timm
!pip3 install torchmetrics
!pip3 install albumentations

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision.transforms as T
from torch import optim
import pandas as pd
import numpy as np
from tqdm import tqdm
from tensorboardX import SummaryWriter
from timm.utils import AverageMeter
import random
import torchmetrics

File paths to the data

In [ ]:
# Kaggle
annotation_file_path = "../input/sorghum-id-fgvc-9/train_cultivar_mapping.csv"
img_dir_path = "../input/sorghum-id-fgvc-9/train_images"

# DeepNote
# annotation_file_path = "data/train_cultivar_mapping.csv"
# img_dir_path = "data/train_images"

In [ ]:
import pandas as pd
import albumentations as A
from pathlib import Path
from albumentations.core.composition import Compose, OneOf
from albumentations.pytorch import ToTensorV2

class DatasetParams:
    img_size = 512
    test_size = 0.2 # change to 0.2 later
    filepaths = {
        'train': Path("./") / "train.csv",
        'test': Path("./") / "test.csv"
    }
    imgdirpath = img_dir_path

    # mean and std of each color channel
    # below is that of imagenet 
    norm_mean = [0.485, 0.456, 0.406]
    norm_std = [0.229, 0.224, 0.225]

DP = DatasetParams

DP.universe = pd.read_csv(Path("../input/sorghum-id-fgvc-9/train_cultivar_mapping.csv"))
DP.universe = DP.universe[DP.universe.image != '.DS_Store'] # sanitize inputs and remove the .DS_Store bulls**t

DP.transforms = {
    'train': Compose([
                A.RandomResizedCrop(height=DP.img_size, width=DP.img_size),
                A.Flip(p=0.5),
                A.RandomRotate90(p=0.5),
                A.ShiftScaleRotate(p=0.5),
                A.HueSaturationValue(p=0.5),
                A.OneOf([
                    A.RandomBrightnessContrast(p=0.5),
                    A.RandomGamma(p=0.5),
                ], p=0.5),
                A.OneOf([
                    A.Blur(p=0.1),
                    A.GaussianBlur(p=0.1),
                    A.MotionBlur(p=0.1),
                ], p=0.1),
                A.OneOf([
                    A.GaussNoise(p=0.1),
                    A.ISONoise(p=0.1),
                    A.GridDropout(ratio=0.5, p=0.2),
                    A.CoarseDropout(max_holes=16, min_holes=8, max_height=16, max_width=16, min_height=8, min_width=8, p=0.2)
                ], p=0.2),
                A.Normalize(
                    mean=DP.norm_mean,
                    std=DP.norm_std,
                ),
                ToTensorV2(),
            ]),
    
    'test': Compose([
                A.Resize(height=DP.img_size, width=DP.img_size),
                A.Normalize(
                    mean=DP.norm_mean,
                    std=DP.norm_std,
                ),
                ToTensorV2(),
            ])
}

DP.transforms_vis = {
    'train': Compose([
                A.RandomResizedCrop(height=DP.img_size, width=DP.img_size),
                A.Flip(p=0.5),
                A.RandomRotate90(p=0.5),
                A.ShiftScaleRotate(p=0.5),
                A.HueSaturationValue(p=0.5),
                A.OneOf([
                    A.RandomBrightnessContrast(p=0.5),
                    A.RandomGamma(p=0.5),
                ], p=0.5),
                A.OneOf([
                    A.Blur(p=0.1),
                    A.GaussianBlur(p=0.1),
                    A.MotionBlur(p=0.1),
                ], p=0.1),
                A.OneOf([
                    A.GaussNoise(p=0.1),
                    A.ISONoise(p=0.1),
                    A.GridDropout(ratio=0.5, p=0.2),
                    A.CoarseDropout(max_holes=16, min_holes=8, max_height=16, max_width=16, min_height=8, min_width=8, p=0.2)
                ], p=0.2),
                ToTensorV2(),
            ]),
    
    'test': Compose([
                A.Resize(height=DP.img_size, width=DP.img_size),
                ToTensorV2(),
            ])
}

Split data into train and test 

In [ ]:
from pathlib import Path
from sklearn.model_selection import train_test_split
import pandas as pd
import random

img_labels = DP.universe
print(len(img_labels))
print(img_labels)

# quick macro for dividing a database into two disjunct ones based on a function that returns boolean
def filter(dataframe, key, lb, yes, no):
    for i, k in enumerate(dataframe[key].tolist()):
        try:
            if lb(k):
                yes.loc[len(yes)] = dataframe.iloc[i]
            else:
                if no is not None:
                    no.loc[len(no)] = dataframe.iloc[i]
        except KeyError:
            raise KeyError(f'Failed at {k}, {i}')
            

train = pd.DataFrame(columns=['image', 'cultivar'])
test = pd.DataFrame(columns=['image', 'cultivar'])
img_to_split = pd.DataFrame(columns=['image', 'cultivar'])

# isolate all class with only 1 training sample and put them in train
counts = img_labels['cultivar'].value_counts()
print(counts)
filter(img_labels, 'cultivar', lambda cultivar: counts[cultivar] == 1, train, img_to_split)


imgs = img_to_split['image'].tolist()
labels = img_to_split['cultivar'].tolist()

print(len(imgs), len(labels))

# use train_test_split with stratify to split class with multiple training samples
train_split, test_split = train_test_split(imgs, test_size=DP.test_size, stratify=labels)
# print('train', train_split)
# print('test_split', test_split, len(test_split))

filter(img_to_split, 'image', lambda image: image in train_split, train, test)

# print('train', train)
# print('test', test)

assert(len(img_labels) == len(train) + len(test))

train.to_csv(DP.filepaths['train'], index=False)
test.to_csv(DP.filepaths['test'], index=False)

Define the dataset

In [ ]:
import os
import pandas as pd
import torch
from torch.utils.data import Dataset
from torchvision.io import read_image
from torchvision.transforms.functional import convert_image_dtype
import cv2
from sklearn import preprocessing

class CultivarDataset(Dataset):
    def __init__(self, annotations_file, img_dir, transform=None, target_transform=None):
        self.img_labels = pd.read_csv(annotations_file)
        self.img_dir = img_dir
        self.transform = transform
        self.target_transform = target_transform

        self.labelenc = preprocessing.LabelEncoder()
        self.labelenc.fit(self.img_labels['cultivar'].tolist())      

    def to_label(self, string: str):
        return self.labelenc.transform([string])[0]  

    def __len__(self):
        return len(self.img_labels)

    def __getitem__(self, idx):
        img_path = os.path.join(self.img_dir, self.img_labels.iloc[idx, 0])

        # NOTE: when pytorch reads an image, it is immediately transformed into a uint8 Tensor with each channel ranging in [0, 255]
        image = cv2.imread(img_path)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

        label = self.to_label(self.img_labels.iloc[idx, 1])

        if self.transform:
            image = self.transform(image=image)['image']
        if self.target_transform:
            label = self.target_transform(label)
        
        return image, label

# small test to make sure it's working as intended
c = CultivarDataset(
    annotations_file=DP.filepaths['train'],
    img_dir=DP.imgdirpath
)

print(c.to_label('PI_257599'))
print(c[0])
print(f'Make sure there is no garbage data: {c.img_labels[c.img_labels["image"].str.contains("2017")==False]}')
    

In [ ]:
datasets = {
    'train': CultivarDataset(
                annotations_file=DP.filepaths['train'],
                img_dir=DP.imgdirpath,
                transform=DP.transforms['train']
            ),
    'test': CultivarDataset(
                annotations_file=DP.filepaths['test'],
                img_dir=DP.imgdirpath,
                transform=DP.transforms['test']
            )
}

print(datasets['train'][0])
print(datasets['test'][0])

In [ ]:
data = pd.read_csv(annotation_file_path)
data.cultivar.value_counts().hist()

Notes on Hyperparameter:
- https://arxiv.org/pdf/1803.09820.pdf
- Momentum is usually always 0.9

In [ ]:
class CustomEffNet(nn.Module):
    def __init__(self, model_name='tf_efficientnet_b0_ns', pretrained=True):
        super().__init__()
        self.model = timm.create_model(model_name, pretrained=pretrained)
        in_features = self.model.get_classifier().in_features
#         self.model.fc = nn.Linear(in_features, CFG.num_classes)
        self.model.classifier = nn.Sequential(
            nn.Linear(in_features, in_features),
            nn.ReLU(inplace=True),
            nn.Dropout(0.5),
            nn.Linear(in_features, 100)
        )

    def forward(self, x):
        x = self.model(x)
        return x

In [ ]:
# Hyperparameters
lr = 1e-4
max_lr = 1e-3
momentum = 0.9
weight_decay = 1e-5
epoches = 25
batch_size = 8
temperature = 0.1
exp_name = 'save_9'

In [ ]:
def set_random_seed(seed=0, deterministic=False):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    if deterministic:
        torch.backends.cudnn.deterministic = True
        torch.backends.cudnn.benchmark = False

In [ ]:
def cultivar_train(model, data_loader_train, optimizer, scheduler, criterion, metric, epoch, summary_writer):
    model.train()
    optimizer.zero_grad()
    loss_meter = AverageMeter()
    acc_meter = AverageMeter()
    # total_samples, correct = 0, 0
    with tqdm(enumerate(data_loader_train), total=len(data_loader_train)) as pbar:
        for idx, (samples, targets) in pbar:
            optimizer.zero_grad()
            samples = samples.cuda()
            targets = targets.cuda()
            out = model(samples)
            loss = criterion(out, targets)
            loss.backward()
            optimizer.step()
            scheduler.step()
            loss_meter.update(loss.item(), targets.size(0))

            # Calculate accuracy
            pred = F.softmax(out)
            acc = metric(pred.argmax(1).cpu(), targets.cpu())
            acc_meter.update(acc, 1)
            # _, predicted = torch.max(pred.data, 1)
            # total_samples += targets.size(0)
            # correct += (predicted == targets).sum().item()
            if idx%10==0:
                summary_writer.add_scalar(f'lr', optimizer.param_groups[0]['lr'], epoch*len(data_loader_train)+idx)
            pbar.set_description(f"Train epoch {epoch}, loss: {loss: .4f}, accuracy: {acc: .4f}, lr: {optimizer.param_groups[0]['lr']: .6f}")

    return loss_meter.avg, acc_meter.avg

In [ ]:
@torch.no_grad()
def cultivar_val(model, data_loader_val, criterion, metric, epoch):
    model.eval()
    loss_meter = AverageMeter()
    acc_meter = AverageMeter()
    # total_samples, correct = 0, 0
    with tqdm(enumerate(data_loader_val), total=len(data_loader_val)) as pbar:
        for idx, (samples, targets) in pbar:
            samples = samples.cuda()
            targets = targets.cuda()
            out = model(samples)
            loss = criterion(out, targets)
            loss_meter.update(loss.item(), targets.size(0))

            # Calculate accuracy
            pred = F.softmax(out)
            acc = metric(pred.argmax(1).cpu(), targets.cpu())
            acc_meter.update(acc, 1)
            # _, predicted = torch.max(pred.data, 1)
            # total_samples += targets.size(0)
            # correct += (predicted == targets).sum().item()
            pbar.set_description(f"Validation epoch {epoch}, loss: {loss: .4f}, accuracy: {acc: .4f}")
    
    return loss_meter.avg, acc_meter.avg

You need to run the `test.csv` and `train.csv` generation in `cultivardataset.ipynb` before running this!


The general procedure of k-fold is as follows:

- Shuffle the dataset randomly.
- Split the dataset into k groups
- For each unique group:
    - Take the group as a hold out or test data set
    - Take the remaining groups as a training data set
    - Fit a model on the training set and evaluate it on the test set
    - Retain the evaluation score and discard the model
- Summarize the skill of the model using the sample of model evaluation scores

In [ ]:
import timm
from timm.loss import SoftTargetCrossEntropy, LabelSmoothingCrossEntropy

set_random_seed(seed=42)
print("Creating datasets...")

summary_writer = SummaryWriter(exp_name)

print("Validation dataset created")
data_loader_train = torch.utils.data.DataLoader(
    datasets['train'],
    batch_size = batch_size,
    num_workers = 2,
    shuffle=True,
    pin_memory = True,
    drop_last = True
)

data_loader_val = torch.utils.data.DataLoader(
    datasets['test'],
    batch_size = batch_size,
    num_workers = 2,
    shuffle=False,
    pin_memory = True,
    drop_last = False
)

print("Dataloader created")
print("Creating model...")
model = CustomEffNet(model_name='tf_efficientnet_b5_ns', pretrained=True)
model.cuda()
optimizer = torch.optim.Adam(model.parameters(), lr=lr)
scheduler = torch.optim.lr_scheduler.OneCycleLR(optimizer, 
                                                        epochs=epoches, steps_per_epoch=len(data_loader_train),
                                                        max_lr=max_lr, pct_start=0.2, 
                                                        div_factor=1e3, final_div_factor=1e3)
criterion = LabelSmoothingCrossEntropy(smoothing=temperature)
metric = torchmetrics.Accuracy(threshold=0.5, num_classes=100)

min_loss = float('inf')

print("Start Training")
for epoch in range(epoches):
    loss_train, acc_train = cultivar_train(model, data_loader_train, optimizer, scheduler, criterion, metric, epoch, summary_writer)
    loss_val, acc_val = cultivar_val(model, data_loader_val, criterion, metric, epoch)
    min_loss = min(min_loss, loss_val)
    if min_loss == loss_val:
        save_state = {'model': model.state_dict(),
                'optimizer': optimizer.state_dict(),
                'epoch': epoch
                }

        save_path = f'ckpt_epoch_{epoch}.pth'
        print(f"{save_path} saving...")
        torch.save(save_state, save_path)
        print(f"{save_path} saved")

    print("Writing to summarywriter...")
    summary_writer.add_scalar(f'Loss/train', loss_train, epoch)
    summary_writer.add_scalar(f'Loss/val', loss_val, epoch)
    summary_writer.add_scalar(f'Acc/train', acc_train, epoch)
    summary_writer.add_scalar(f'Acc/val', acc_val, epoch)
    summary_writer.add_scalar(f'Min_loss', min_loss, epoch)

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=d841b2e3-7f2f-42e6-ae8e-6cea1c0a3631' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>